In [1]:
import oommfc as mc
import discretisedfield as df
import micromagneticmodel as mm

region = df.Region(p1=(-50e-9, -50e-9, 0), p2=(50e-9, 50e-9, 10e-9))
subregions = {'r1': df.Region(p1=(-50e-9, -50e-9, 0), p2=(0, 50e-9, 10e-9)),
              'r2': df.Region(p1=(0, -50e-9, 0), p2=(50e-9, 50e-9, 10e-9))}
mesh = df.Mesh(region=region, cell=(5e-9, 5e-9, 5e-9), subregions=subregions)

system = mm.System(name='fs')

system.energy = mm.Zeeman(H=(0, 0, 1e6))
system.dynamics = mm.Precession(gamma0=mm.consts.gamma0) + mm.Damping(alpha=0.1)

Ms = 1.1e6

# create system with above geometry and initial magnetisation
system.m = df.Field(mesh, dim=3, value=(1, 0, 0), norm=Ms)

In [2]:
# minimize the energy
td = mc.TimeDriver()
td.drive(system, t=2e-9, n=10, fixed_subregions=['r2'])

# md = mc.MinDriver()
# md.drive(system, fixed_spins=['r1'])

# Plot relaxed configuration: vectors in z-plane
system.m.plane('z').mpl()

NameError: name 'n_threads' is not defined

In [ ]:
system.m.orientation((20e-9, 0, 0))

In [ ]:
system.table.mpl(yaxis=['E_zeeman'])

In [ ]:
system.table.mpl(yaxis=['mz'])

In [ ]:
system.table

In [ ]:
import micromagneticmodel as mm

In [ ]:
mm.__version__